# Face Recognition Using mlrun with OpenCV And scikit-learn
 A complete pipeline of data processing, model training and serving function deployment.

### Install mlrun and kubeflow pipelines

In [ ]:
!pip install mlrun
!pip install kfp

### Restart jupyter kernel after initial installations
Either restart manually, or run the following code cell. 

In [ ]:
import os
os._exit(00)

### Install dependencies for the code and set config 

In [3]:
# nuclio: ignore
import nuclio

Change following magic command to %%nuclio cmd -c if the following packages are already installed locally.

In [ ]:
%%nuclio cmd
pip install cmake
pip install dlib
pip install face_recognition
pip install opencv-contrib-python
pip install imutils
pip install sklearn 
pip install pandas
pip install joblib
pip install v3io_frames

In [ ]:
%nuclio config spec.build.baseImage = "python:3.6-jessie"

### Declare global variables and perform necessary imports 

In [4]:
DATA_PATH = '/User/demos/face-recognition/dataset/'
ARTIFACTS_PATH = '/User/demos/face-recognition/artifacts/'

In [5]:
import os
import shutil
import zipfile
from urllib.request import urlopen
from io import BytesIO
import cv2
import face_recognition
from imutils import paths
import joblib
from sklearn.linear_model import LogisticRegression
from mlrun.artifacts import TableArtifact
import pandas as pd
import numpy as np
import datetime
import random
import string
import v3io_frames as v3f

### Import and define mlrun functions for the pipeline 

In [6]:
# nuclio: ignore
from mlrun import new_function, code_to_function, NewTask, mount_v3io
import kfp
from kfp import dsl

In [7]:
def encode_images(context):
    
    client = v3f.Client("framesd:8081", container="users")
    
    # If no train images exist in the predefined path we will train the model on a small dataset of movie actresses
    if not os.path.exists(DATA_PATH + 'input'):
        os.makedirs(DATA_PATH + 'input')
        resp = urlopen('https://iguazio-public.s3.amazonaws.com/roy-actresses/Actresses.zip')
        zip_ref = zipfile.ZipFile(BytesIO(resp.read()), 'r')
        zip_ref.extractall(DATA_PATH + 'input')
        zip_ref.close()
          
    
    idx_file_path = ARTIFACTS_PATH+"idx2name.csv"
    if os.path.exists(idx_file_path):
        idx2name_df = pd.read_csv(idx_file_path)
    else:
        idx2name_df = pd.DataFrame(columns=['value', 'name'])
    
    
    #creates a mapping of classes(person's names) to target value
    new_classes_names = [f for f in os.listdir(DATA_PATH + 'input') if not '.ipynb' in f and f not in idx2name_df['name'].values]
    
    initial_len = len(idx2name_df)
    final_len = len(idx2name_df) + len(new_classes_names)
    for i in range(initial_len, final_len):
        idx2name_df.loc[i] = {'value': i, 'name': new_classes_names.pop()}
        
    name2idx = idx2name_df.set_index('name')['value'].to_dict()
            
    
    #log name to index mapping into mlrun context
    context.log_artifact(TableArtifact('idx2name', df=idx2name_df), src_path='idx2name.csv')
    
    #generates a list of paths to labeled images 
    imagePaths = [f for f in paths.list_images(DATA_PATH + 'input') if not '.ipynb' in f]
    knownEncodings = []
    knownLabels = []
    fileNames = []
    urls = []
    for (i, imagePath) in enumerate(imagePaths):
        print("[INFO] processing image {}/{}".format(i + 1, len(imagePaths)))
        #extracts label (person's name) of the image
        name = imagePath.split(os.path.sep)[-2]
        
        #prepares to relocate image after extracting features
        file_name = imagePath.split(os.path.sep)[-1]
        new_path = DATA_PATH + 'processed/' + file_name 
        
        #converts image format to RGB for comptability with face_recognition library
        image = cv2.imread(imagePath)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        #detects coordinates of faces bounding boxes
        boxes = face_recognition.face_locations(rgb, model='hog')
        
        #computes embeddings for detected faces
        encodings = face_recognition.face_encodings(rgb, boxes)
        
        #this code assumes that a person's folder in the dataset does not contain an image with a face other then his own
        for enc in encodings:
            file_name = name + '_' + ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))                                                            
            knownEncodings.append(enc)
            knownLabels.append([name2idx[name]])
            fileNames.append(file_name)
            urls.append(new_path)
        
        #move image to processed images directory
        shutil.move(imagePath, new_path)
        
    #saves computed encodings to avoid repeating computations
    df_x = pd.DataFrame(knownEncodings, columns=['c' + str(i).zfill(3) for i in range(128)]).reset_index(drop=True)
    df_y = pd.DataFrame(knownLabels, columns=['label']).reset_index(drop=True)
    df_details = pd.DataFrame([['initial training']*3]*len(df_x), columns=['imgUrl', 'camera', 'time'])
    df_details['time'] = [datetime.datetime.utcnow()]*len(df_x)
    df_details['imgUrl'] = urls
    data_df = pd.concat([df_x, df_y, df_details], axis=1)
    data_df['fileName'] = fileNames
    
    client.write(backend='kv', table='iguazio/demos/face-recognition/artifacts/encodings', dfs=data_df, index_cols=['fileName'])
    with open('artifacts/encodings_path.txt', 'w+') as f:
        f.write('iguazio/demos/face-recognition/artifacts/encodings')
    context.log_artifact('encodings_path', src_path=f.name)

In [8]:
def train(context, processed_data, model_name='model.bst'):
    #trains classifier
    context.logger.info('Client')
    client = v3f.Client("framesd:8081", container="users" )
    with open(processed_data.url, 'r') as f:
        t = f.read()
    
    data_df = client.read(backend="kv", table=t, reset_index=False)
    
    X_train = data_df[['c'+str(i).zfill(3) for i in range(128)]].values
    y_train = data_df['label'].values
    
    model = LogisticRegression(multi_class='ovr', solver='lbfgs').fit(X_train, y_train)
    
    context.logger.info('Save model')
    #saves and logs model into mlrun context
    joblib.dump(model, model_name)
    context.log_artifact('model', src_path=model_name, labels={'framework': 'sklearn_classifier'})

In [9]:
# nuclio: end-code

In [10]:
# serving function
serving_function = code_to_function(name='recognize-faces', 
                                      filename='./nuclio-face-prediction.ipynb',
                                      runtime='nuclio')
serving_function.with_http(workers=2).add_volume('User','~/')

### Test pipeline functions locally

In [11]:
task = NewTask(handler=encode_images, out_path=ARTIFACTS_PATH)
run = new_function().run(task)

[mlrun] 2019-12-02 14:05:45,721 starting run encode_images uid=8a3ae21f24914f488f1198f74bede3d4  -> 


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...ede3d4,0,Dec 02 14:05:45,completed,encode_images,kind=handlerowner=iguaziohost=jupyter-omympcgsri-dfhpb-6c4cc55b44-5hdb4,,,,idx2nameencodings_path


type result.show() to see detailed results/progress or use CLI:
!mlrun get run --uid 8a3ae21f24914f488f1198f74bede3d4 
[mlrun] 2019-12-02 14:05:46,031 run executed, status=completed


In [ ]:
task2 = NewTask(handler=train, inputs={'processed_data': run.outputs['encodings_path']}, out_path=ARTIFACTS_PATH)
train = new_function().run(task2)

### Create a function from notebook and build image
supposed to take a few minutes

In [ ]:
fn = code_to_function('face-recognition')

In [ ]:
fn.build(image='mlrun/face_recognition:latest')

### Create pipeline

In [ ]:
@dsl.pipeline(
    name='face recognition pipeline',
    description='Creates and deploys a face recognition model'
)
def face_recognition_pipeline():
    fn.with_code()
    
    encode = fn.as_step(name='encode-images', handler='encode_images', out_path=ARTIFACTS_PATH, outputs=['idx2name', 'encodings_path']).apply(mount_v3io())
    
    train = fn.as_step(name='train', handler='train', out_path=ARTIFACTS_PATH, outputs=['model'], 
                               inputs={'processed_data': encode.outputs['encodings_path']}).apply(mount_v3io())
    
    deploy = serving_function.deploy_step(project='default', models={'face_rec_v1': train.outputs['model']})
    

In [ ]:
client = kfp.Client(namespace='default-tenant')

In [ ]:
#For debug purposes compile pipeline code
kfp.compiler.Compiler().compile(face_recognition_pipeline, 'face_rec.yaml')

### Run pipeline

In [ ]:
arguments = {}
run_result = client.create_run_from_pipeline_func(face_recognition_pipeline, arguments=arguments, run_name='face_rec_1', experiment_name='face_rec')